# Property Based Testing in Go

## What is property based testing
* rather than testing based on examples, we find properties that should hold for a piece of functionality.
* to prove these properties hold, we then generate lots of sample data to test against the property

## Advantages of property based testing
* Much larger input scope covered ~> gives confidence that edge cases are also covered 

## Examples of common properties

- Associative – The order in which opperations are applied doesn't matter
    - Example:
        - a \* (b \* c) == (a \* b) \* c


- Commutative – The order in which arguments are applied doesn't matter
    - Example:
        - a + b == b + a    


- Distributive
    - Example:
        - a \* (b + c) = ab + ac
        - title.ToUpper() + author.ToUpper() = (title + author).ToUpper()


- Idempotent – applying a function any amount of times will yield the same result
    - Example:
        - terraform apply == terraform apply `and then` terraform apply

- Identity – There is an identity value that you can apply to any other value to return the same value [a . i == a]
    - Example:
        - a + 0 == a
        - a * 1 == a


- Bilbo Property (aka, There And Back Again) - You can get back to the original state
    - Example: 
        - list.Reverse().Reverse() == list
        - a == db.Set(a) `and then` db.Get(a)

## An example

In [8]:
func Add(a, b int) int {
    return a + b
}

## How can we test this?
* we could give it some example scenarios
    * we have to know the answers to the scenarios before hand
    * we end up with a very small input set
    * is saying that 1+2=3 and 3+4=7 really describing what addition is?

## What do we know about addition?
* Commutative - a + b == b + a
* Associative - (a + b) + c == a + (b + c)
* Identity - a + 0 == a

In [9]:
import (   
    "github.com/leanovate/gopter"
    "github.com/leanovate/gopter/gen"
    "github.com/leanovate/gopter/prop"
)

In [10]:
properties := gopter.NewProperties(nil)

In [11]:
properties.Property("Adding is commutative", prop.ForAll(
    func(a, b int) bool {
        return Add(a, b) == Add(b, a)
    },
    gen.Int(), gen.Int(),
))

In [12]:
properties.Property("Adding is associative", prop.ForAll(
    func(a, b, c int) bool {
        return Add(a, Add(b, c)) == Add(Add(a, b), c)
    },
    gen.Int(), gen.Int(), gen.Int(),
))

In [13]:
properties.Property("Adding 0 will always yield the same result", prop.ForAll(
    func(v int) bool {
        return Add(v, 0) == v 
    },
    gen.Int(),
))

In [14]:
properties.Run(gopter.ConsoleReporter(false))

+ Adding is commutative: OK, passed 100 tests.
+ Adding is associative: OK, passed 100 tests.
+ Adding 0 will always yield the same result: OK, passed 100 tests.


true

In [59]:
func Add(a, b int) int {
    return a * b
}

In [60]:
properties.Run(gopter.ConsoleReporter(false))

+ Adding is commutative: OK, passed 100 tests.
Elapsed time: 675.4µs
+ Adding is associative: OK, passed 100 tests.
Elapsed time: 1.3393ms
! Adding 0 will always yield the same result: Falsified after 0 passed
   tests.
ARG_0: 1
ARG_0_ORIGINAL (30 shrinks): 563953682
Elapsed time: 457.7µs


false

**Note:** It's probably still a good idea to have a couple of examples along with these tests. Property based tests shouldn't be complementary to example based tests

## A real world example

### A document store

In [294]:
type DocumentStore interface {
    Add(identifier string, data string)
    Read(identifier string) (string, bool)
}

In [295]:
import "unicode/utf8"

type CrapDB struct {
    data map[string]string
}

func (s CrapDB) Add(identifier string, data string) {
    if val, ok := s.data[identifier]; ok {
        panic("failed to insert: identifier already in use")
        return 
    }
    
    if utf8.RuneCountInString(identifier) > 36 {
        panic("failed to insert: your identifier should not to be longer than 36 characters (the length of a uuid)")
    }
    
    s.data[identifier] = data
}

func (s CrapDB) Read(identifier string) (string, bool) {
    return s.data[identifier]
}

In [296]:
type StorageService struct {
    db DocumentStore
}

func NewStorageService(db DocumentStore) *StorageService {
    return &StorageService{
        db: db,
    }
}

func (s *StorageService) Add(identifier string, data string) {
    s.db.Add(identifier, data)
}

func (s *StorageService) Read(identifier string) (string, bool) {
    return s.db.Read(identifier)
}

In [297]:
import "github.com/leanovate/gopter/arbitrary"

properties := gopter.NewProperties(nil)
arbitraries := arbitrary.DefaultArbitraries()

properties.Property("Getting an inserted entry should return that entry", arbitraries.ForAll(
    func(identifier string, data string, db CrapDB) string {
        store := NewStorageService(db)
        
        store.Add(identifier, data)
        storedData, _ := store.Read(identifier)
        
        if storedData != data {
            return "stored data is " + storedData + ", expected " + data
        }
        return ""
    },
))

In [299]:
properties.Run(gopter.ConsoleReporter(false))

! Getting an inserted entry should return that entry: Error on property
   evaluation after 2 passed tests: Check paniced: failed to insert:
   identifier already in use
ARG_0: 
ARG_1: 񎷲
ARG_2: {𒀸data:map[:񲨊]}


false

In [267]:
type StorageService struct {
    db DocumentStore
}

func NewStorageService(db DocumentStore) *StorageService {
    return &StorageService{
        db: db,
    }
}

func (s *StorageService) Add(identifier string, data string) bool {
    _, exists := s.db.Read(identifier)
    
    if !exists {
        s.db.Add(identifier, data)
        return true
    }
    
    return false
}

func (s *StorageService) Read(identifier string) (string, bool) {
    return s.db.Read(identifier)
}

In [288]:
properties := gopter.NewProperties(nil)
arbitraries := arbitrary.DefaultArbitraries()

properties.Property("Getting an inserted entry should return that entry", arbitraries.ForAll(
    func(identifier string, data string, db CrapDB) string {
        store := NewStorageService(db)
        
        ok := store.Add(identifier, data)
        if !ok {
            return ""
        }
        
        storedData, _ := store.Read(identifier)
        
        if storedData != data {
            return "stored data is " + storedData + ", expected " + data
        }
        return ""
    },
))

In [289]:
properties.Run(gopter.ConsoleReporter(false))

! Getting an inserted entry should return that entry: Error on property
   evaluation after 43 passed tests: Check paniced: failed to insert: your
   identifier should not to be longer than 36 characters (the length of a
   uuid)
ARG_0:
   񭞁񱃾􀨲󟩍뵓󴣾􉢇󞷆򴖾򈀴򨐛򡘏𬦐󌬂򲲼󴊩򞒳󋦢���
  �񉒾񢕺𪪋󊒻󛼮󏓝񊤬񱅑󜅉􅒌𲙡󆅆񼵑򂲃𚘮򷲋𥚶𹒃
ARG_0_ORIGINAL (2 shrinks):
   򔪯񭞁񱃾􀨲󟩍뵓󴣾􉢇󞷆򴖾򈀴򨐛򡘏𬦐󌬂򲲼󴊩򞒳���
  �󻶚񉒾񢕺𪪋󊒻󛼮󏓝񊤬񱅑󜅉􅒌𲙡󆅆񼵑򂲃𚘮򷲋𥚶��
  ��󵚌
ARG_1: 󞢼
ARG_1_ORIGINAL (6 shrinks):
   񖌑𺨆𑌿񤺐󕯍񏲞񏄹񲭀󌖈򀏴򺫵񝊶𒃨𵅍𭵼󀾈򯇔򴫫��
  ��򣬨𛿖򛻀񛼏􇍜𿐣򠢖񷍀򣎦򪁖󪪠𿏍󘥸򑰒񂪯񴮛󽰩򥔚�
  ���𭻹򣨡󳻡󞢼
ARG_2:
   {𒀸data:map[𹒃:񖌑𺨆𑌿񤺐󕯍񏲞񏄹񲭀󌖈򀏴򺫵񝊶𒃨���
  �𭵼󀾈򯇔򴫫򲩏򣬨𛿖򛻀񛼏􇍜𿐣򠢖񷍀򣎦򪁖󪪠𿏍󘥸��
  ��񂪯񴮛󽰩򥔚􄆁𭻹򣨡󳻡󞢼]}
ARG_2_ORIGINAL (13 shrinks):
   {𒀸data:map[󈠅󋓪񓁵𦌽򪑩:󺪗玤񇿌򕯖򨀬饧򛄐񵠣󉓿򃈯�
  ���𴄵𥣒𑣩򥄋󻜸򇶳򘿄󱇦󻋇憹󱔇񒪼󼂖듶򁎭𗕻񍓁衉���
  �
   򔪯񭞁񱃾􀨲󟩍뵓󴣾􉢇󞷆򴖾򈀴򨐛󴊩򞒳󋦢󻶚񉒾񢕺���
  �󊒻󛼮󏓝񊤬񱅑󜅉􅒌𲙡󆅆񼵑򂲃𚘮򷲋𥚶𹒃:񖌑𺨆𑌿�
  ���󕯍񏲞񏄹񲭀󌖈򀏴򺫵񝊶𒃨𵅍𭵼󀾈򯇔򴫫򲩏򣬨𛿖򛻀
  񛼏􇍜𿐣򠢖񷍀򣎦򪁖󪪠𿏍󘥸򑰒񂪯񴮛󽰩򥔚􄆁𭻹򣨡���
  �󞢼
   񭞁񱃾􀨲󟩍뵓󴣾􉢇󞷆򴖾򈀴򨐛򡘏𬦐󌬂򲲼󴊩򞒳󋦢���
  �񉒾񢕺𪪋󛼮󏓝񊤬񱅑󜅉􅒌𲙡󆅆񼵑򂲃𚘮򷲋𥚶𹒃:񖌑�
  ���𑌿񤺐󕯍񏲞񏄹񲭀󌖈򀏴򺫵񝊶𒃨𵅍𭵼󀾈򯇔򴫫򲩏򣬨
  𛿖򛻀񛼏􇍜𿐣򠢖񷍀򣎦򪁖󪪠𿏍󘥸򑰒񂪯񴮛󽰩򥔚􄆁���
  �򣨡󳻡󞢼
   򘰗񔵍󿪲򙉻􉘈񲱅񘩗񨣼򶉆𜥭𖩔򀯹䴒󇞔򺚒🺾󫘊򎽈
  񄃕𚌗򣪶񨷹󷬼򙄞𾩶󆨇򾪸򱖓񛕪󂱱񃆭뗸򩰍򽕚󫶂𸩋:���
  �򤆄󞈩𖢴𵁸񬲥𜔉񠖘􅆆󃰡
   򔪯񭞁񱃾􀨲󟩍뵓󴣾􉢇󞷆򴖾򈀴򨐛򡘏𬦐󌬂򲲼󴊩򞒳���
  �󻶚񉒾񢕺𪪋󊒻󛼮󏓝񊤬񱅑󜅉􅒌𲙡󆅆𥚶𹒃:񖌑𺨆𑌿�
  ���󕯍񏲞񏄹񲭀󌖈򀏴򺫵񝊶𒃨𵅍𭵼󀾈򯇔򴫫򲩏򣬨𛿖򛻀
  񛼏􇍜𿐣򠢖񷍀򣎦򪁖󪪠𿏍󘥸򑰒񂪯񴮛󽰩򥔚􄆁𭻹򣨡���
  

  ���𑌿񤺐󕯍񏲞񏄹񲭀󌖈򀏴򺫵񝊶𒃨𵅍𭵼󀾈򯇔򴫫򲩏򣬨
  𛿖򛻀񛼏􇍜𿐣򠢖񷍀򣎦򪁖󪪠𿏍󘥸򑰒񂪯񴮛󽰩򥔚􄆁���
  �򣨡󳻡󞢼
   򔪯񭞁񱃾􀨲󟩍뵓󴣾􉢇󞷆󋦢󻶚񉒾񢕺𪪋󊒻󛼮󏓝񊤬���
  �󜅉􅒌𲙡󆅆񼵑򂲃𚘮򷲋𥚶𹒃:񖌑𺨆𑌿񤺐󕯍񏲞񏄹񲭀�
  ���򀏴򺫵񝊶𒃨𵅍𭵼󀾈򯇔򴫫򲩏򣬨𛿖򛻀񛼏􇍜𿐣򠢖񷍀
  򣎦򪁖󪪠𿏍󘥸򑰒񂪯񴮛󽰩򥔚􄆁𭻹򣨡󳻡󞢼
   򔪯񭞁񱃾􀨲󟩍뵓󴣾􉢇󞷆򴖾򈀴򨐛򡘏𬦐󴊩򞒳󋦢󻶚���
  �񢕺𪪋󊒻󛼮󏓝񊤬񱅑󜅉􅒌𲙡󆅆񼵑򂲃𚘮򷲋𥚶𹒃:񖌑�
  ���𑌿񤺐󕯍񏲞񏄹񲭀󌖈򀏴򺫵񝊶𒃨𵅍𭵼󀾈򯇔򴫫򲩏򣬨
  𛿖򛻀񛼏􇍜𿐣򠢖񷍀򣎦򪁖󪪠𿏍󘥸򑰒񂪯񴮛󽰩򥔚􄆁���
  �򣨡󳻡󞢼
   񭞁񱃾􀨲󟩍뵓󴣾􉢇󞷆򴖾򈀴򨐛򡘏𬦐󌬂򲲼󴊩򞒳󋦢���
  �􅒌𲙡󆅆񼵑򂲃𚘮򷲋𥚶𹒃:񖌑𺨆𑌿񤺐󕯍񏲞񏄹񲭀󌖈�
  ���򺫵񝊶𒃨𵅍𭵼󀾈򯇔򴫫򲩏򣬨𛿖򛻀񛼏􇍜𿐣򠢖񷍀򣎦
  򪁖󪪠𿏍󘥸򑰒񂪯񴮛󽰩򥔚􄆁𭻹򣨡󳻡󞢼
   񭞁􀨲󟩍뵓󴣾􉢇󞷆򴖾򈀴򨐛򡘏𬦐󌬂򲲼󴊩򞒳󋦢󻶚���
  �񢕺𪪋󊒻󛼮󏓝񊤬񱅑󜅉􅒌𲙡󆅆񼵑򂲃𚘮򷲋𥚶𹒃:񖌑�
  ���𑌿񤺐󕯍񏲞񏄹񲭀󌖈򀏴򺫵񝊶𒃨𵅍𭵼󀾈򯇔򴫫򲩏򣬨
  𛿖򛻀񛼏􇍜𿐣򠢖񷍀򣎦򪁖󪪠𿏍󘥸򑰒񂪯񴮛󽰩򥔚􄆁���
  �򣨡󳻡󞢼
   񭞁񱃾􀨲󟩍뵓󴣾􉢇󞷆򴖾򨐛򡘏𬦐󌬂򲲼󴊩򞒳󋦢󻶚���
  �񢕺𪪋󊒻󛼮󏓝񊤬񱅑󜅉􅒌𲙡󆅆񼵑򂲃𚘮򷲋𥚶𹒃:񖌑�
  ���𑌿񤺐󕯍񏲞񏄹񲭀󌖈򀏴򺫵񝊶𒃨𵅍𭵼󀾈򯇔򴫫򲩏򣬨
  𛿖򛻀񛼏􇍜𿐣򠢖񷍀򣎦򪁖󪪠𿏍󘥸򑰒񂪯񴮛󽰩򥔚􄆁���
  �򣨡󳻡󞢼
   񭞁񱃾􀨲󟩍뵓󴣾􉢇󞷆򴖾򈀴򨐛򡘏𬦐󌬂򲲼󴊩򞒳󋦢���
  �񉒾񢕺𪪋󊒻󛼮󏓝񊤬񱅑󜅉􅒌𲙡񼵑򂲃𚘮򷲋𥚶𹒃:񖌑�
  ���𑌿񤺐󕯍񏲞񏄹񲭀󌖈򀏴򺫵񝊶𒃨𵅍𭵼󀾈򯇔򴫫򲩏򣬨
  𛿖򛻀񛼏􇍜𿐣򠢖񷍀򣎦򪁖󪪠𿏍󘥸򑰒񂪯񴮛󽰩򥔚􄆁���
  �򣨡󳻡󞢼
   󆯬񣒱󝂿񵎆򸚩𙶘𧦢񌓦𒫁􎀭򕫷񓞭맑񌝱񮱊󗗀𰼯񯕳���
  �󊑳򱞶𫧔󡑇𝛜񷨅򷥢󡲦񘌴񯍻񈋃񨴴𸂋:񜊇񧏀񻯥𙼿��
  �𺀊
   򔪯񭞁񱃾􀨲󟩍뵓󴣾􉢇󞷆򴖾򈀴򨐛򡘏𬦐󌬂򲲼󋦢󻶚���
  �񢕺𪪋󊒻󛼮󏓝񊤬񱅑󜅉􅒌𲙡󆅆񼵑򂲃𚘮򷲋𥚶𹒃:񖌑�
  ���𑌿񤺐󕯍񏲞񏄹񲭀󌖈򀏴򺫵񝊶𒃨𵅍𭵼󀾈򯇔򴫫򲩏򣬨
  𛿖򛻀񛼏􇍜𿐣򠢖񷍀򣎦򪁖󪪠𿏍󘥸򑰒񂪯񴮛󽰩򥔚􄆁���
  �򣨡󳻡󞢼
   򔪯񭞁񱃾􀨲󟩍뵓󴣾􉢇󞷆򴖾򈀴򨐛򡘏𬦐󌬂򲲼󴊩򞒳���
  �󻶚񉒾񢕺𪪋󊒻񊤬񱅑󜅉􅒌𲙡󆅆񼵑򂲃𚘮򷲋𥚶𹒃:񖌑�
  ���𑌿񤺐󕯍񏲞񏄹񲭀󌖈򀏴򺫵񝊶𒃨𵅍𭵼󀾈򯇔򴫫򲩏򣬨
  𛿖򛻀񛼏􇍜𿐣򠢖񷍀򣎦򪁖󪪠𿏍󘥸򑰒񂪯񴮛󽰩򥔚􄆁���
  �򣨡󳻡󞢼
   񭞁񱃾􀨲󟩍뵓󴣾􉢇󞷆򴖾򈀴򨐛򡘏𬦐󌬂󴊩򞒳󋦢󻶚���
  �񢕺𪪋󊒻󛼮󏓝񊤬񱅑󜅉􅒌𲙡󆅆񼵑򂲃𚘮򷲋𥚶𹒃:񖌑�
  ���𑌿񤺐󕯍񏲞񏄹񲭀󌖈򀏴򺫵񝊶𒃨𵅍𭵼󀾈򯇔򴫫򲩏򣬨
  𛿖򛻀񛼏􇍜𿐣򠢖񷍀򣎦򪁖󪪠

false

In [291]:
type StorageService struct {
    db DocumentStore
}

func NewStorageService(db DocumentStore) *StorageService {
    return &StorageService{
        db: db,
    }
}

func (s *StorageService) Add(identifier string, data string) bool {
    _, exists := s.db.Read(identifier)
    
    // since db.Add seems to panic, we should probably use recover (try/catch) to return our `ok`
    if !exists && len(identifier) < 36 {
        s.db.Add(identifier, data)
        return true
    }
    
    return false
}

func (s *StorageService) Read(identifier string) (string, bool) {
    return s.db.Read(identifier)
}

In [292]:
properties.Run(gopter.ConsoleReporter(false))

+ Getting an inserted entry should return that entry: OK, passed 100 tests.


true

## What did we learn about CrapDB (that we might have missed with examples)?
* Inserting to an existing key panics
* ids can't be too long

* **we weren't able to imagine all of the edge cases**

## Why I chose to use gopter

* supports shrinking
* allows you to create multiple generators for the same type
* generators are composable
* library maintainer is active and contributing PRs is straight forward

## What have property based tests caught so far

* unhandled nil pointers
* storage size limits
    * exceeding MySQL varchar length
    * exceeding DynamoDB max row size
* cases not covered by example based test
    * bug where JOIN on a Jet query only returned ever returned a list of one item
* inconsistent inputs and outputs when inserting into DynamoDB
    * When inserting int64s we would get back float64s back
    * inserting an empty map doesn't insert anything into DynamoDB so you can't check if a key exists on an empty map

## Other things to note
* The more the tests run through the pipeline, the more different scenarios will be run.

* since the seed will be different every time. On a failed test, the seed is printed so that **scenarios that rarely occur are reproduceable**

## Links
- https://dev.to/jdsteinhauser/intro-to-property-based-testing-2cj8
- https://github.com/leanovate/gopter